In [ ]:
#復習問題
#分析対象の文字列
text = 'Ich bin sehr zufrieden.'

In [ ]:
# pers_create_dataset
import boto3
import json
import time

personalize = boto3.client('personalize')

# これ以降書く
personalize = boto3.client('personalize')
schema_json = {
    'type': 'record',
    'name': 'Interactions',
    'namespace': 'com.anazonaws.personalize.schema',
    'fields': [
        {
            'name': 'USER_ID',
            'type': 'string'
        },
        {
            'name': 'TEN_ID',
            'type': 'string'
        },
        {
            'name': 'TINESTAWP',
            'type': 'long'
        }
    ],
    'version': '1.0'
}

schema_arn = personalize.create_schema(
    name='MySchema', schema=json.dumps(schema_json))['schemaArn']
print('schema ARN:', schema_arn)
group_arn = personalize.create_dataset_group(
    name='MyGroup')['datasetGroupArn']
print('dataset group ARN:', group_arn)

start = time.time()
status = ''
while status not in ['ACTIVE', 'CREATE FAILED']:
    status = personalize.describe_dataset_group(datasetGroupArn=group_arn)[
        'datasetGroup']['status']
    time.sleep(10)
    print('{:7.2f} {}'.format(time.time() - start, status))
dataset_arn = personalize.create_dataset(
    name='MyDataset', datasetType='interactions',
    schemaArn=schema_arn, datasetGroupArn=group_arn)['datasetArn']
print('dataset ARN:', dataset_arn)


In [ ]:
#pers_create_role
import boto3
import json

iam = boto3.client('iam')

role_json = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {
                'Service': 'personalize.amazonaws.com'
            },
            'Action': 'sts:AssumeRole'
        }
    ]
}
role = iam.create_role(
    RoleName='PersonalizeRole',
    AssumeRolePolicyDocument=json.dumps(role_json))
print('role ARN:', role['Role']['Arn'])

policy_json = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Action': [
                's3:ListBucket',
                's3:GetObject'
            ],
            'Effect': 'Allow',
            'Resource': [
                'arn:aws:s3:::*'
            ]
        }
    ]
}
policy = iam.create_policy(
    PolicyName='PersonalizePolicy',
    PolicyDocument=json.dumps(policy_json))
print('policy ARN:', policy['Policy']['Arn'])

iam.attach_role_policy(
    RoleName='PersonalizeRole',
    PolicyArn=policy['Policy']['Arn'])

In [ ]:
# pers_import_dataset
import boto3
import json
import uuid
import time

region = 'ap-northeast-1'
s3 = boto3.client('s3', region)

bucket = str(uuid.uuid1())
print('bucket:', bucket)
s3.create_bucket(
    Bucket=bucket,
    CreateBucketConfiguration={'LocationConstraint': region})
policy_json = {
    'Version': '2012-10-17',
    'Id': 'PersonalizeS3BucketAccessPolicy',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {
                'Service': 'personalize.amazonaws.com'
            },
            'Action': [
                's3:ListBucket',
                's3:GetObject'
            ],
            'Resource': [
                'arn:aws:s3:::'+bucket,
                'arn:aws:s3:::'+bucket+'/*'
            ]
        }
    ]
}
s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy_json))
file_name = 'interaction.csv'
s3.upload_file(file_name, bucket, file_name)

iam = boto3.client('iam')
role_arn = iam.get_role(RoleName='PersonalizeRole')['Role']['Arn']

# これ以降書く（(8)データセットグループ(MyGroup)を取得から）
personalize = boto3.client('personalize')

for group in personalize.list_dataset_groups()['datasetGroups']:
    if group['name'] == 'MyGroup':
        break
group_arn = group['datasetGroupArn']
for dataset in personalize.list_datasets(
        datasetGroupArn=group_arn)['datasets']:
    if dataset['name'] == 'MyDataset':
        break
dataset_arn = dataset['datasetArn']

job_arn = personalize.create_dataset_import_job(
    jobName='MyJob', datasetArn=dataset_arn,
    dataSource={'datalocation': 's3://'+ bucket + '/'+ file_name},
    roleArn=role_arn)['datasetImportJobArn']
print('job ARN:', job_arn)

start = time.time()
status = ''
while status not in ['ACTIVE', 'CREATE FAILED']:
    status = personalize.describe_dataset_import_job(
        datasetImportJobArn=job_arn)['dataset ImportJob']['status']
    time.sleep(10)
    print('{:7.2f} {}'.format(time.time()-start, status))


In [ ]:
# pers_create_solution
import boto3
import time

personalize = boto3.client('personalize')

for group in personalize.list_dataset_groups()['datasetGroups']:
    if group['name'] == 'MyGroup':
        break
group_arn = group['datasetGroupArn']

solution_arn = personalize.create_solution(
    name='MySolution', datasetGroupArn=group_arn, performAutoML=True
)['solutionArn']
print('solution ARN', solution_arn)

start = time.time()
status = ''
while status not in ['ACTIVE', 'CREATE FAILED']:
    status = personalize.describe_solution(solutionArn=solution_arn)[
        'solution']['status']
    time.sleep(10)
    print('{:7.2f} {}'.format(time.time()-start, status))
version_arn = personalize.create_solution_version(
    solutionArn=solution_arn)['solutionVersionArn']
print('solution version ARN:', version_arn)

start = time.time()
status = ''
while status not in ['ACTIVE', 'CREATE FAILED']:
    status = personalize.describe_solution_version(solutionVersionArn=version_arn)[
        'solutionVersion']['status']
    time.sleep(10)
    print('{:7.2f} {}'.format(time.time()-start, status))


In [ ]:
# pers_create_campaign
import boto3
import time

personalize = boto3.client('personalize')

for group in personalize.list_dataset_groups()['datasetGroups']:
    if group['name'] == 'MyGroup':
        break
group_arn = group['datasetGroupArn']

# これ以降書く（(2)ソリューション(MySolution)を取得から）
for solution in personalize.list_solutions(datasetGroupArn=group_arn)['solutions']:
    if solution['name'] == 'MySolution':
        break
solution_arn = solution['solutionArn']

version_arn = personalize.list_solution_versions(solutionArn=solution_arn)[
    'solutionVersions'][0]['solutionVersionArn']

campaign_arn = personalize.create_campaign(
    name='MyCampaign', solutionVersionArn=version_arn, minProvisionedTPS=1)['campaignArn']

print('campaign ARN:', campaign_arn)

start = time.time()
status = ''
while status not in ['ACTIVE', 'CREATE FAILED']:
    status = personalize.describe_campaign(campaignArn=campaign_arn)[
        'campaign']['status']
    time.sleep(10)
    print('{:7.2f} {}'.format(time.time()-start, status))


In [ ]:
# pers_recommend
import boto3
import sys

userid = input('ユーザIDを入力してください：')

personalize = boto3.client('personalize')
# これ以降書く（※テキストで「sys.argv[1]」の部分は「userid」に書き換える）
personalize_runtime = boto3.client('personalize-runtime')

for group in personalize.list_dataset_groups()['datasetGroups']:
    if group['name'] == 'MyGroup':
        break
group_arn = group['datasetGroupArn']
for solution in personalize.list_solutions(datasetGroupArn=group_arn)['solutions']:
    if solution['name'] == 'MySolution':
        break
solution_arn = solution['solutionArn']

for campaign in personalize.list_campaigns(solutionArn=solution_arn)['campaigns']:
    if campaign['name'] == 'MyCampaign':
        break
campaign_arn = campaign['campaignArn']

result = personalize_runtime.get_recommendations(
    campaignArn=campaign_arn, userId=userid)

with open('item.txt', 'r', encoding='UTF-8') as file:
    name = list(file)

for rank, item in enumerate(result['itemList'], 1):
    print('{:2} {}'.format(rank, name[int(item['itemId'])]), end='')


In [ ]:
#pers_delete_campaign
import boto3

personalize = boto3.client('personalize')

for campaign in personalize.list_campaigns()['campaigns']:
    campaign_arn = campaign['campaignArn']
    print('campaign ARN:', campaign_arn)
    personalize.delete_campaign(campaignArn=campaign_arn)

In [ ]:
#pers_delete_solution
import boto3

personalize = boto3.client('personalize')

for solution in personalize.list_solutions()['solutions']:
    solution_arn = solution['solutionArn']
    print('solution ARN:', solution_arn)
    personalize.delete_solution(solutionArn=solution_arn)

In [ ]:
#pers_delete_role
import boto3

iam = boto3.client('iam')

for policy in iam.list_policies()['Policies']:
    if policy['PolicyName'] == 'PersonalizePolicy':
        break
policy_arn = policy['Arn']
print('policy ARN:', policy_arn)

iam.detach_role_policy(
    RoleName='PersonalizeRole', PolicyArn=policy_arn)
iam.delete_policy(PolicyArn=policy_arn)
iam.delete_role(RoleName='PersonalizeRole')

In [ ]:
#pers_delete_dataset
import boto3
import time

personalize = boto3.client('personalize')

for dataset in personalize.list_datasets()['datasets']:
    dataset_arn = dataset['datasetArn']
    print('dataset ARN:', dataset_arn)
    personalize.delete_dataset(datasetArn=dataset_arn)

for group in personalize.list_dataset_groups()['datasetGroups']:
    group_arn = group['datasetGroupArn']
    print('dataset group ARN:', group_arn)
    while True:
        try:
            personalize.delete_dataset_group(
                datasetGroupArn=group_arn)
        except Exception:
            time.sleep(1)
            break

for schema in personalize.list_schemas()['schemas']:
    schema_arn = schema['schemaArn']
    print('schema ARN:', group_arn)
    while True:
        try:
            personalize.delete_schema(schemaArn=schema_arn)
        except Exception:
            time.sleep(1)
            break